# Mini RAG - Abordagem Procedural Passo a Passo

Este notebook demonstra como criar um sistema RAG (Retrieval-Augmented Generation) completo usando uma abordagem **procedural** e **didática**.

## O que é RAG?

RAG é uma técnica que combina:
- **Recuperação de informações** (busca em documentos)
- **Geração de texto** (usando um LLM)

Com RAG, o LLM pode responder perguntas baseadas em documentos específicos (como PDFs, manuais, etc.) ao invés de apenas usar seu conhecimento geral.

## Fluxo Completo do RAG

```
1. 📄 Carregar documentos (PDFs)
2. ✂️  Dividir em chunks (pedaços menores)
3. 🧮 Criar embeddings (vetores numéricos)
4. 💾 Armazenar no banco vetorial (FAISS)
5. 🔍 Buscar chunks relevantes
6. 📝 Formatar contexto
7. 💬 Criar prompt
8. 🤖 Gerar resposta com LLM
```

Vamos seguir cada passo!

## Passo 1: Importar Bibliotecas

Primeiro, importamos todas as bibliotecas necessárias para construir nosso sistema RAG.

In [60]:
import os
import requests
from pathlib import Path
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS

from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader



## Passo 2: Configurar Ambiente

Definimos as configurações básicas:
- URL do Ollama (servidor local de LLMs)
- Caminho para os PDFs que serão processados

In [61]:
# Configurações iniciais (parâmetros simples)
OLLAMA_BASE_URL = 'http://localhost:11434'
BASE_DIR = Path(__file__).parent if "__file__" in globals() else Path.cwd()
PDF_DIR = BASE_DIR.parent.parent / "data" / "pdfs"

# Modelos
EMBEDDING_MODEL = 'embeddinggemma'
LLM_MODEL = 'llama3.2:1b'

print(f"📁 Diretório de PDFs: {PDF_DIR}")
print(f"🤖 Ollama URL: {OLLAMA_BASE_URL}")

📁 Diretório de PDFs: e:\01-projetos\11-work\11.34-engenharia-vetorial\data\pdfs
🤖 Ollama URL: http://localhost:11434


In [62]:
# Checagem mínima de ambiente (caminho feliz)
print('Verificando Ollama...')
resp = requests.post(f"{OLLAMA_BASE_URL}/api/show")
print('Ollama status:', resp.status_code)
print('Modelo de embedding:', EMBEDDING_MODEL)
print('Modelo LLM:', LLM_MODEL)

# Import opcional para exibição
from IPython.display import Markdown, display
import pandas as pd

Verificando Ollama...
Ollama status: 400
Modelo de embedding: embeddinggemma
Modelo LLM: llama3.2:1b


## Passo 3: Carregar Documentos (PDFs)

**O que acontece aqui?**
- Localizamos todos os arquivos PDF na pasta `data/pdfs/`
- Carregamos cada PDF usando o `PyPDFLoader`
- Cada página do PDF vira um documento separado

**Resultado:** Uma lista de documentos, onde cada documento = 1 página de PDF

In [63]:
# Lê os documentos
documents = []

pdf_paths = list(PDF_DIR.glob("*.pdf"))
print(f"📚 Encontrados {len(pdf_paths)} PDFs\n")

for pdf_path in pdf_paths:
    loader = PyPDFLoader(str(pdf_path))
    docs = loader.load()
    documents.extend(docs)
    print(f"  ✓ {pdf_path.name}: {len(docs)} páginas")
    
print(f"📄 Total de documentos carregados: {len(documents)}")

incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams


📚 Encontrados 4 PDFs

  ✓ api_documentation.pdf: 3 páginas
  ✓ livro_receitas.pdf: 5 páginas
  ✓ manual_futebol.pdf: 4 páginas
  ✓ manual_iphone15.pdf: 3 páginas
📄 Total de documentos carregados: 15


## Passo 4: Dividir em Chunks (Chunking)

**Por que fazer isso?**
- Páginas de PDF podem ser muito grandes
- LLMs têm limite de tokens
- Chunks menores = buscas mais precisas

**Configurações:**
- `chunk_size=1000`: Cada chunk terá ~1000 caracteres
- `chunk_overlap=200`: Overlap de 200 chars entre chunks consecutivos (evita cortar frases importantes)
- `separators`: Prioriza quebras em parágrafos (`\n\n`), depois linhas (`\n`), depois espaços

**Resultado:** Uma lista de chunks (pedaços) menores e mais gerenciáveis

In [64]:
# Parâmetros de chunking / busca
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200
SEPARATORS = ["\n\n", "\n", " ", ""]

print(f"🔢 Parâmetros: chunk_size={CHUNK_SIZE}, chunk_overlap={CHUNK_OVERLAP}")

# Divide os documentos em chunks (usa parâmetros do topo)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    separators=SEPARATORS,
)

chunks = text_splitter.split_documents(documents=documents)

print(f"✂️  Total de chunks criados: {len(chunks)}")
print(f"📊 Expansão de chunking: {len(chunks)/len(documents):.1f}x")

🔢 Parâmetros: chunk_size=1000, chunk_overlap=200
✂️  Total de chunks criados: 33
📊 Expansão de chunking: 2.2x


## Passo 5: Criar Embeddings

**O que são embeddings?**
- São vetores numéricos (listas de números) que representam o significado do texto
- Textos com significados similares têm vetores similares

**Modelo usado:** `embeddinggemma` (Modelo do Google, roda localmente via Ollama)

**Por que isso é importante?**
- Permite buscar documentos por **significado**, não apenas por palavras-chave
- Exemplo: "como fazer café" encontrará textos sobre "preparar bebida quente" mesmo sem a palavra "café"

In [65]:
# Cria os embeddings dos chunks (usa EMBEDDING_MODEL simples)
embeddings = OllamaEmbeddings(
    model=EMBEDDING_MODEL, 
    base_url=OLLAMA_BASE_URL
)

print(f"🧠 Modelo de embeddings: {EMBEDDING_MODEL}")
print("✅ Embeddings configurados!")

🧠 Modelo de embeddings: embeddinggemma
✅ Embeddings configurados!


## Passo 6: Armazenar no Banco Vetorial (FAISS)

**O que é FAISS?**
- Biblioteca da Meta (Facebook) para busca eficiente de vetores
- Armazena todos os embeddings e permite buscas rápidas

**O que acontece aqui?**
1. Cada chunk é convertido em embedding (vetor)
2. Todos os vetores são indexados no FAISS
3. Agora podemos fazer buscas por similaridade!

**Analogia:** É como criar um índice de biblioteca, mas baseado no **significado** ao invés de ordem alfabética

In [66]:
# Cria vectorstore (caminho feliz)
print("⏳ Criando índice FAISS (isso pode demorar alguns minutos...)\n")

vectorstore = FAISS.from_documents(
    documents=chunks, 
    embedding=embeddings
)

print(f"\n✅ Vectorstore criado!")
print(f"📊 Total de vetores indexados: {vectorstore.index.ntotal}")
print(f"📐 Dimensões dos embeddings: {vectorstore.index.d}")

⏳ Criando índice FAISS (isso pode demorar alguns minutos...)


✅ Vectorstore criado!
📊 Total de vetores indexados: 33
📐 Dimensões dos embeddings: 768


In [78]:
# Cria o retriever (modo simples)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
print("🔍 Retriever criado (k=4)")

🔍 Retriever criado (k=4)


## Passo 7: Buscar Documentos Relevantes (Retrieval)

**O coração do RAG!**

Aqui fazemos a **busca semântica**:
1. Transformamos a pergunta em embedding (vetor)
2. Comparamos com todos os embeddings dos chunks
3. Retornamos os `k=4` chunks mais similares

**Métrica de similaridade:** Distância cosseno (quanto menor, mais similar)

**Resultado:** Os 4 chunks que melhor respondem à pergunta

In [68]:
# Query de teste (caminho feliz usando retriever quando disponível)
query_futebol = "Quais são as principais formações táticas do futebol?"

# Usa retriever.invoke se disponível, senão fallback para similarity_search
if hasattr(retriever, 'invoke'):
    relevant_documents = retriever.invoke(query_futebol)
    relevant_scores = [None] * len(relevant_documents)
else:
    relevant_documents = vectorstore.similarity_search(query_futebol, k=K)
    relevant_scores = [None] * len(relevant_documents)

print(f"✅ Documentos retornados: {len(relevant_documents)}")
# Exibe em formato tabular
rows = []
for d in relevant_documents:
    rows.append({'source': d.metadata.get('source', 'N/A'), 'page': d.metadata.get('page', 'N/A'), 'snippet': d.page_content[:200].replace('\n', ' ')})
df = pd.DataFrame(rows)
display(df)

✅ Documentos retornados: 4


,source,page,snippet
0,e:\01-projetos\11-work\11.34-engenharia-vetori...,2,Formações Táticas Clássicas 4-4-2 (Formação Eq...
1,e:\01-projetos\11-work\11.34-engenharia-vetori...,3, Mantém organização espacial do time  Dificu...
2,e:\01-projetos\11-work\11.34-engenharia-vetori...,2,Formação ofensiva popularizada pelo Barcelona ...
3,e:\01-projetos\11-work\11.34-engenharia-vetori...,3,4-2-3-1 (Formação Moderna de Controle): Sistem...


## Passo 8: Formatar o Contexto

**Por que fazer isso?**
- O LLM precisa receber os chunks em formato de texto
- Juntamos os 4 chunks recuperados com `\n\n` (quebra de parágrafo)

**Resultado:** Um único texto contendo todo o contexto relevante para responder à pergunta

In [79]:
# Formata os documentos (usa os relevant_documents obtidos acima)
formatted_documents = '\n\n'.join(doc.page_content for doc in relevant_documents)

# Passa os documentos formatados para o contexto
context = formatted_documents
question = query_futebol

# Mostra o prompt gerado (preview)
prompt_result = prompt.invoke({'context': context, 'question': question})
text = getattr(prompt_result, 'text', str(prompt_result))
clean = text.replace('\x7f', '•')
display(Markdown(f"```text\n{clean[:1000]}\n```"))

```text
Use o seguinte contexto para responder à pergunta.
Se você não souber a resposta baseado no contexto, diga "Não encontrei essa informação nos documentos fornecidos."

Importante: Cite sempre a fonte (nome do arquivo e página) quando possível.

Contexto:
Formações Táticas Clássicas
4-4-2 (Formação Equilibrada Clássica):
A formação mais tradicional e equilibrada do futebol moderno.
Estrutura:
• 4 defensores: 2 laterais (direito e esquerdo) + 2 zagueiros centrais
• 4 meio-campistas: 2 alas (direita e esquerda) + 2 volantes/meias centrais
• 2 atacantes: dupla de centroavantes
Características:
• Boa cobertura defensiva com linha de 4 zagueiros
• Meio-campo povoado permite controle do jogo
• Dupla de ataque facilita cruzamentos e jogadas aéreas
• Laterais têm liberdade para apoiar o ataque
• Compacta: distância curta entre linhas (ideal 10-12 metros)
Variação 4-4-2 losango:
• 1 volante de contenção
• 2 meias laterais
• 1 meia armador (ponta do losango)
• Mais controle no meio, menos largura

```

## Passo 9: Criar o Prompt

**O que é um prompt?**
- É a instrução que damos ao LLM
- Define como o LLM deve responder

**Elementos do nosso prompt:**
1. **Instrução:** "Use o seguinte contexto para responder"
2. **Regra:** "Se não souber, diga que não encontrou"
3. **Contexto:** Os chunks recuperados
4. **Pergunta:** A pergunta do usuário

**Template pattern:** Usamos `{context}` e `{question}` como placeholders que serão preenchidos depois

In [70]:

# Template de prompt customizado
template = """Use o seguinte contexto para responder à pergunta.
Se você não souber a resposta baseado no contexto, diga "Não encontrei essa informação nos documentos fornecidos."

Importante: Cite sempre a fonte (nome do arquivo e página) quando possível.

Contexto:
{context}

Pergunta: {question}

Resposta detalhada:"""

# Criação do PromptTemplate utilizando o template acima
prompt = PromptTemplate(template=template, input_variables=['context', 'question'])

print("📝 Prompt template criado!")



📝 Prompt template criado!


### Preencher o Prompt

Agora substituímos os placeholders `{context}` e `{question}` pelos valores reais:
- `context` = os chunks formatados
- `question` = a pergunta do usuário

**Resultado:** Um prompt completo, pronto para ser enviado ao LLM

In [71]:
from IPython.display import Markdown, display

# Passa context e question para o prompt e retorna o resultado
prompt_result = prompt.invoke({'context': context, 'question': question})

text = getattr(prompt_result, "text", str(prompt_result))
clean = text.replace("\\x7f", "•")     # substitui caractere de controle por símbolo visível
display(Markdown(f"```text\n{clean}\n```"))

```text
Use o seguinte contexto para responder à pergunta.
Se você não souber a resposta baseado no contexto, diga "Não encontrei essa informação nos documentos fornecidos."

Importante: Cite sempre a fonte (nome do arquivo e página) quando possível.

Contexto:
Formações Táticas Clássicas
4-4-2 (Formação Equilibrada Clássica):
A formação mais tradicional e equilibrada do futebol moderno.
Estrutura:
 4 defensores: 2 laterais (direito e esquerdo) + 2 zagueiros centrais
 4 meio-campistas: 2 alas (direita e esquerda) + 2 volantes/meias centrais
 2 atacantes: dupla de centroavantes
Características:
 Boa cobertura defensiva com linha de 4 zagueiros
 Meio-campo povoado permite controle do jogo
 Dupla de ataque facilita cruzamentos e jogadas aéreas
 Laterais têm liberdade para apoiar o ataque
 Compacta: distância curta entre linhas (ideal 10-12 metros)
Variação 4-4-2 losango:
 1 volante de contenção
 2 meias laterais
 1 meia armador (ponta do losango)
 Mais controle no meio, menos largura
4-3-3 (Formação Ofensiva com Wingers):
Formação ofensiva popularizada pelo Barcelona e Liverpool.
Estrutura:
 4 defensores: mesma linha do 4-4-2
 3 meio-campistas: 1 volante + 2 meias (ou 2 volantes + 1 meia)

 Mantém organização espacial do time
 Dificulta movimentação adversária
Marcação Individual:
 Cada defensor marca um atacante específico
 Segue o adversário por todo o campo
 Usado em bolas paradas
 Arriscado: pode ser desorganizado com movimentações
Marcação Pressão:
 Pressionar adversário com a bola imediatamente
 Reduzir tempo e espaço para pensar
 Forçar erros e recuperar bola no campo de ataque
 Exige condicionamento físico excepcional
 Popularizado por Klopp (Gegenpressing)
Transições Rápidas (Contra-ataque):
 Velocidade na passagem defesa-ataque
 Aproveitar desorganização adversária
 Lançamentos longos ou passes rápidos
 Jogadores velozes nas pontas
Posse de Bola (Tiki-Taka):
 Manter posse com passes curtos
 Movimentação constante sem bola
 Triângulos de passe (3 jogadores próximos)
 Cansar adversário e controlar ritmo
 Popularizado pelo Barcelona de Guardiola

Formação ofensiva popularizada pelo Barcelona e Liverpool.
Estrutura:
 4 defensores: mesma linha do 4-4-2
 3 meio-campistas: 1 volante + 2 meias (ou 2 volantes + 1 meia)
 3 atacantes: 1 centroavante + 2 extremos/pontas (abertos)
Características:
 Alta amplitude ofensiva (três atacantes abertos)
 Domínio de posse com triângulos no meio-campo
 Pontas cortam para dentro ou ficam abertos para receber
 Exige laterais com boa condição física (cobrem toda a lateral)
 Volante crucial como "pivô" entre defesa e ataque
Variação 4-3-3 falso 9:
 Centroavante recua para criar espaço
 Pontas infiltram na área deixada pelo 9
 Meia armador sobe para ocupar posição de 10
3-5-2 (Formação com Ala-defensores):
Sistema com três zagueiros e domínio do meio-campo.
Estrutura:
 3 defensores: zagueiro central (líbero) + 2 zagueiros laterais
 5 meio-campistas: 2 alas (wing-backs) + 3 meio-campistas centrais
 2 atacantes: dupla de centroavantes
Características:

4-2-3-1 (Formação Moderna de Controle):
Sistema tático mais usado por seleções nas últimas Copas do Mundo.
Estrutura:
 4 defensores: linha tradicional
 2 volantes: dupla de contenção
 3 meio-atacantes: 2 meias abertos + 1 meia central (camisa 10)
 1 centroavante: referência fixa
Características:
 Dupla de volantes protege a defesa
 Meia central (#10) tem liberdade criativa
 Transições rápidas com meias abertos em velocidade
 Centroavante segura a bola e finaliza
 Compacto defensivamente, fluido no ataque
Varia para 4-4-2 na defesa:
 Meia central recua para linha do meio
 Meias abertos fecham como alas
 Atacante não fica isolado (meia dá apoio)
Conceitos Táticos Modernos:
Marcação por Zona:
 Cada jogador responsável por uma zona do campo
 Marca o adversário que entrar em sua zona
 Mantém organização espacial do time
 Dificulta movimentação adversária
Marcação Individual:
 Cada defensor marca um atacante específico
 Segue o adversário por todo o campo

Pergunta: Quais são as principais formações táticas do futebol?

Resposta detalhada:
```

### Criar o LLM

**Modelo usado:** `llama3.2:1b` (versão compacta do Llama 3.2)

**Por que usar Ollama?**
- Roda **localmente** (sem enviar dados para a nuvem)
- **Gratuito** (sem custos de API)
- **Privado** (seus documentos ficam no seu computador)

O modelo receberá o prompt e gerará a resposta final!

In [72]:
# Cria o LLM (caminho feliz)
llm = OllamaLLM(model=LLM_MODEL, base_url=OLLAMA_BASE_URL)
print(f"🤖 LLM configurado: {LLM_MODEL}")

🤖 LLM configurado: llama3.2:1b


## Passo 10: Gerar a Resposta

**O momento final!**

Aqui o LLM:
1. Recebe o prompt completo (instrução + contexto + pergunta)
2. Analisa os chunks recuperados
3. Gera uma resposta baseada **apenas** no contexto fornecido

**Importante:** A resposta será específica dos seus PDFs, não do conhecimento geral do modelo!

In [73]:
# Invoca o LLM com o prompt completo
response = llm.invoke(prompt_result)

# Exibe a resposta
print("🤖 Resposta do LLM:")
print("=" * 80)
print(response)
print("=" * 80)

🤖 Resposta do LLM:
Com base no contexto fornecido, aqui estão as principais formações táticas do futebol:

1. **4-4-2 (Formação Equilibrada Clássica)**:
   - 4 defensores: 2 laterais + 2 zagueiros centrais
   - 4 meio-campistas: 2 alas + 2 volantes/meias centrais
   - 2 atacantes: dupla de centroavantes

2. **4-3-3 (Formação Ofensiva com Wingers)**:
   - 4 defensores: linha do 4-4-2
   - 3 meio-campistas: volante + meias laterais
   - 2 atacantes: dupla de centroavantes

3. **4-2-3-1 (Formação Moderna de Controle)**:
   - 4 defensores: linha tradicional
   - 2 volantes: dupla de contenção
   - 3 meio-campistas: meias abertos + meia central
   - 1 centroavante

4. **4-4-2 Losango**:
   - 1 volante de contenção
   - 2 meias laterais
   - 1 meia armador (ponta do losango)
   - Maior controle no meio, menos largura

5. **3-5-2**:
   - Sistema com três zagueiros e domínio do meio-campo
   - 2 atacantes: dupla de centroavantes

6. **3-4-3**:
   - Sistema com três zagueiros e domínio dos meia

In [74]:
# Quick Run: executa prompt -> llm e exibe resposta
response = llm.invoke(prompt_result)

print("🤖 Resposta do LLM:")
print("=" * 80)
print(response)
print("=" * 80)

display(Markdown('**Dica:** para versão com chains e streaming veja: [lab_3.4_microrag_chain_lcel.ipynb](lab_3.4_microrag_chain_lcel.ipynb)'))

🤖 Resposta do LLM:
A resposta à sua pergunta sobre as principais formações táticas do futebol inclui várias estruturas e conceitos que variam em termos de número de defensores, meio-campistas e atacantes. Aqui está uma resumo detalhado das principais formações mencionadas:

1. **4-2 (Formação Equilibrada Clássica)**: Esta é a formação mais tradicional e equilibrada do futebol moderno, com 4 defensores, 2 laterais e 2 zagueiros centrais. Caracteriza-se por uma boa cobertura defensiva, meio-campo povoado para controle do jogo e dupla de ataque com jogadores que podem jogar como alas ou volantes.

2. **4-3-3**: Esta formação popularizada pelo Barcelona e Liverpool mantém a organização espacial do time tradicionalmente, com 4 defensores, 3 meio-campistas e 2 atacantes. O sistema de marcação individual permite que cada jogador crie uma zona de controle no campo, enquanto a diferença entre o centroavante e os extremos cria um elemento de pressão nos jogadores adversários.

3. **4-3-1**: Esse

**Dica:** para versão com chains e streaming veja: [lab_3.4_microrag_chain_lcel.ipynb](lab_3.4_microrag_chain_lcel.ipynb)

## Passo 11 (Opcional): Análise das Fontes

Vamos verificar quais chunks foram recuperados e usados como contexto.

In [75]:
# Exibe os documentos recuperados
print("📚 Documentos usados como fonte:\n")

for i, doc in enumerate(relevant_documents, 1):
    print(f"Documento {i}:")
    print(f"Fonte: {doc.metadata.get('source', 'N/A')}")
    print(f"Página: {doc.metadata.get('page', 'N/A')}")
    print(f"Conteúdo (primeiros 200 chars):\n{doc.page_content[:200]}...")
    print("-" * 80)

📚 Documentos usados como fonte:

Documento 1:
Fonte: e:\01-projetos\11-work\11.34-engenharia-vetorial\data\pdfs\manual_futebol.pdf
Página: 2
Conteúdo (primeiros 200 chars):
Formações Táticas Clássicas
4-4-2 (Formação Equilibrada Clássica):
A formação mais tradicional e equilibrada do futebol moderno.
Estrutura:
 4 defensores: 2 laterais (direito e esquerdo) + 2 zagueiro...
--------------------------------------------------------------------------------
Documento 2:
Fonte: e:\01-projetos\11-work\11.34-engenharia-vetorial\data\pdfs\manual_futebol.pdf
Página: 3
Conteúdo (primeiros 200 chars):
 Mantém organização espacial do time
 Dificulta movimentação adversária
Marcação Individual:
 Cada defensor marca um atacante específico
 Segue o adversário por todo o campo
 Usado em bolas parad...
--------------------------------------------------------------------------------
Documento 3:
Fonte: e:\01-projetos\11-work\11.34-engenharia-vetorial\data\pdfs\manual_futebol.pdf
Página: 2
Conteúdo 

## Passo 12 (Opcional): Teste com Outra Pergunta

Agora vamos testar com uma pergunta diferente para ver como o sistema se comporta.

In [76]:
# Nova pergunta
query_receita = "Como fazer uma lasanha?"

# Busca documentos relevantes
relevant_documents_2 = vectorstore.similarity_search(query_receita, k=4)

# Formata contexto
context_2 = '\n\n'.join(doc.page_content for doc in relevant_documents_2)

# Preenche o prompt
prompt_result_2 = prompt.invoke({'context': context_2, 'question': query_receita})

# Gera resposta
response_2 = llm.invoke(prompt_result_2)

# Exibe
print(f"Pergunta: {query_receita}\n")
print("🤖 Resposta:")
print("=" * 80)
print(response_2)
print("=" * 80)

Pergunta: Como fazer uma lasanha?

🤖 Resposta:
Para criar uma lasanha típica, você precisará seguir os passos abaixo. A seguir estão as instruções detalhadas para obter resultados finais.

### Para a lasanha:

#### Ingredientes:
- 500g de massa fresca (ou 600g se estiver usando massa fresca)
- 400g de muçarela ralada
- 200g de presunto fatiado
- 100g de parmesão ralado

### Modo de Preparo:

1. **Preaqueça o forno**: Pré-aqueça o forno a 180°C.
2. **Montagem da lasanha**: 
   - Pré-aqueça um refratário grande (35x25cm) em manteiga, até ficar suave e úmida.  
   - Espalhe 2 conchas de molho bolonhesa no fundo do refratário.
   - Faça a primeira camada de massa para as lasanhas sobrepondo levemente as placas.
   - Espalhe molho bolonhesa, depois molho branco, presunto e muçarela. Repita as camadas: massa, molho branco, presunto, muçarela até completar os ingredientes, finalizando com uma camada de queijos.

3. **Finalize**: Espalhe parmesão ralado por cima.
4. **Leve ao forno**: Cubra co

## 📊 Análise do Sistema RAG

Vamos entender melhor o que nosso sistema criou:

**Métricas importantes:**
- **Expansão de chunking:** Quantos chunks foram criados por página
- **Tamanho dos chunks:** Verifica se o chunking está balanceado
- **Dimensões dos embeddings:** Tamanho dos vetores (maior = mais preciso, mas mais lento)

Essas estatísticas ajudam a **otimizar** o sistema para melhor performance!

In [77]:
# Estatísticas do sistema
print("📊 Estatísticas do Sistema RAG\n")
print("=" * 80)

print(f"Total de PDFs carregados: {len(pdf_paths)}")
print(f"Total de páginas: {len(documents)}")
print(f"Total de chunks criados: {len(chunks)}")
print(f"Expansão de chunking: {len(chunks)/len(documents):.1f}x")

# Tamanho médio dos chunks
tamanhos = [len(c.page_content) for c in chunks]
print(f"\nTamanho médio dos chunks: {sum(tamanhos)/len(tamanhos):.0f} caracteres")
print(f"Tamanho mínimo: {min(tamanhos)} caracteres")
print(f"Tamanho máximo: {max(tamanhos)} caracteres")

# Informações do vectorstore
print(f"\nDimensões dos embeddings: {vectorstore.index.d}")
print(f"Total de vetores no índice: {vectorstore.index.ntotal}")

print("=" * 80)

📊 Estatísticas do Sistema RAG

Total de PDFs carregados: 4
Total de páginas: 15
Total de chunks criados: 33
Expansão de chunking: 2.2x

Tamanho médio dos chunks: 815 caracteres
Tamanho mínimo: 262 caracteres
Tamanho máximo: 997 caracteres

Dimensões dos embeddings: 768
Total de vetores no índice: 33


---

## 🎓 Resumo do que Aprendemos

Parabéns! Você acabou de criar um sistema RAG completo do zero!

### O que fizemos:

1. ✅ **Carregamos PDFs** e transformamos em documentos
2. ✅ **Dividimos em chunks** para melhor processamento
3. ✅ **Criamos embeddings** (vetores de significado)
4. ✅ **Armazenamos no FAISS** para busca eficiente
5. ✅ **Buscamos chunks relevantes** usando similaridade semântica
6. ✅ **Formatamos o contexto** para o LLM
7. ✅ **Criamos um prompt** com instruções claras
8. ✅ **Geramos respostas** baseadas nos documentos

### Por que RAG é importante?

- 📚 **Conhecimento atualizado:** Não precisa retreinar o modelo
- 🎯 **Respostas específicas:** Baseadas nos seus documentos
- 🔒 **Privacidade:** Tudo roda localmente
- 💰 **Custo:** Sem APIs pagas (usando Ollama)

### Próximos passos:

- Experimente com diferentes `chunk_size` e `chunk_overlap`
- Teste outros modelos de embeddings
- Compare modelos de LLM diferentes
- Adicione mais PDFs e veja como o sistema escala

**Dica:** Para produção, considere usar bancos vetoriais mais robustos como Qdrant ou Pinecone!